In [9]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, START, END

from openai import OpenAI
from dotenv import load_dotenv
import os



In [11]:

load_dotenv(override=True)
my_api_key = os.getenv("OPENAI_API_KEY")
# print (f'Key is {my_api_key}')


client = OpenAI(api_key=my_api_key)


In [12]:
# --- Shared State ---
class LibraryState(TypedDict):
    question: Optional[str]
    faq_answer: Optional[str]
    checkout_info: Optional[str]
    final_answer: Optional[str]


def ClassifierAgent(state: LibraryState):
    
    print(f"ClassifierAgent Initial state: {state}")
    
    # state: {'question': 'When does library open?'}
    
    question = state["question"]
    print(f"Classifying question: {question}")

    # Build the LLM messages
    message_to_llm = [
        {"role": "system", "content": '''You are a classifier agent in a library system. 
        Decide if the user is asking about book availability/checkout or about library FAQs. 
        Reply with JSON containing keys: faq_answer and checkout_info.'''},
        {"role": "user", "content": f"Question: {question}"}
    ]
    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=message_to_llm,
        
    )
    print (response)

    # langchain_chain = prompt | message_to_llm | response_processor


    # Extract the content from the response
    answer = response.choices[0].message.content
    
    print (f"ClassifierAgent function state: {state}")

    # Ideally, parse as JSON — here assuming model returns a dict-like string

    try:
        import json
        parsed = json.loads(answer)
        print(f"Raw response: {answer}")
        print(f"Parsed response: {parsed}")

        return {
            "faq_answer": parsed.get("faq_answer", ""), #parsed["faq_answer"]
            "checkout_info": parsed.get("checkout_info", "")
        }
    except Exception:
        # fallback if LLM gives plain text
        return {"faq_answer": answer, "checkout_info": ""}


In [13]:
def FAQAgent(state: LibraryState):
    
    print(f"FAQAgent Initial state: {state}")
    if not state.get("faq_answer"):
        return {"faq_answer": "Default FAQ: Library rules apply"}
    return {"faq_answer": state["faq_answer"]}


def CheckoutAgent(state: LibraryState):
    print(f"CheckoutAgent Initial state: {state}")
    if not state.get("checkout_info"):
        return {"checkout_info": "Checkout info: Not requested"}
    return { "checkout_info": state["checkout_info"]}


def ResponseAgent(state: LibraryState):
    print(f"CheckouResponseAgent Initial state: {state}")
  
    message_to_llm = [
        {"role": "system", "content": '''You are a response builder for the library application.
          Please combine the FAQ answer and checkout info into a coherent response to the user's question.
        '''},
        {"role": "user", "content": f"FAQ: {state['faq_answer']}\nCheckout Info: {state['checkout_info']}\nQuestion: {state['question']}"}
    ]

    # Call the OpenAI model
    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=message_to_llm,
     
    )

    print (response)
    # Extract the content from the response
    final_answer = response.choices[0].message.content


    return {"final_answer": final_answer}


In [14]:
# --- Build the Graph ---
builder = StateGraph(LibraryState)
builder.add_node("ClassifierAgent", ClassifierAgent)
builder.add_node("FAQAgent", FAQAgent)
builder.add_node("CheckoutAgent", CheckoutAgent)
builder.add_node("ResponseAgent", ResponseAgent)

builder.add_edge(START, "ClassifierAgent")
builder.add_edge("ClassifierAgent", "FAQAgent")
builder.add_edge("ClassifierAgent", "CheckoutAgent")
builder.add_edge("FAQAgent", "ResponseAgent")
builder.add_edge("CheckoutAgent", "ResponseAgent")
builder.add_edge("ResponseAgent", END)

graph = builder.compile()

In [15]:
result = graph.invoke({"question": "Is The new best seller available? When does library open?"})
print("\n--- Final Answer ---")
print(result["final_answer"])

ClassifierAgent Initial state: {'question': 'Is The new best seller available? When does library open?'}
Classifying question: Is The new best seller available? When does library open?
ChatCompletion(id='chatcmpl-CmPex0cNpcxBfEMnhUXJTm20b5zX3', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "faq_answer": "Opening hours vary by day. For today\'s hours, please check the library’s hours page on our website or ask staff.",\n  "checkout_info": "To check if \'The new best seller\' is available: search the catalog (by title) on the library website or ask a librarian. If it’s available, you can check it out with your library card. If it’s currently checked out, you can place a hold/reservation to be notified when it becomes available."\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1765654415, model='gpt-5-nano-2025-08-07', object='chat.completion', service_tier='default

In [16]:

# # --- Visualize ---
# print(graph.get_graph().draw_ascii())
# graph.get_graph().draw_png("images/agentic_ai_library.png")
# print("Graph saved as agentic_ai_library.png")

In [17]:
# --- Run ---
result = graph.invoke({"question": "When does Cupertino library open? I would like to check out a book as well."})
print("\n--- Final Answer ---")
print(result["final_answer"])

ClassifierAgent Initial state: {'question': 'When does Cupertino library open? I would like to check out a book as well.'}
Classifying question: When does Cupertino library open? I would like to check out a book as well.
ChatCompletion(id='chatcmpl-CmPgaet0SbvJa08iFyRrIv8mw3oWN', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "faq_answer": "Cupertino Library hours vary by day and season. To get today’s hours, please check the Cupertino Library page on the library’s website or contact the library directly.",\n  "checkout_info": "To check out a book, you’ll need a valid library card (Cupertino/Santa Clara County Library District). If you don’t have one, you can apply online or in person. Steps: 1) Find the book in the catalog (online or in the library). 2) If it’s available, check out at a self-checkout station or ask a staff member to assist. 3) If it’s checked out, place a hold and pick it up later when it’s available. 4) Borro

In [18]:
result = graph.invoke({"question": "Is The Hobbit available? When does the library open"})
print("\n--- Final Answer ---")
print(result["final_answer"])

ClassifierAgent Initial state: {'question': 'Is The Hobbit available? When does the library open'}
Classifying question: Is The Hobbit available? When does the library open
ChatCompletion(id='chatcmpl-CmPh3Br28esSfUHAB250nvvdNlo1k', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "faq_answer": "Opening hours vary by branch. Please specify your branch to get exact opening times, or check the hours on the library\'s website or in the online catalog.",\n  "checkout_info": "To check if \'The Hobbit\' is available for checkout, search the library\'s online catalog or ask a librarian. If you tell me your branch (e.g., Downtown, Northside), I can guide you to the right steps or help you place a hold."\n}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1765654545, model='gpt-5-nano-2025-08-07', object='chat.completion', service_tier='default', system_fingerprint=None, usage=C